In [ ]:
import pandas as pd

In [ ]:
df_featurized_gencode = pd.read_csv("data/1-rmv_dup_introns_gencode_v44.tsv", sep="\t")
df_featurized_gencode

In [ ]:
# Read the original tsv file uploaded into the GSoC ML project repo to extract the rejected introns
df_original_test = pd.read_csv("ftp-data/gene_annot_test.tsv", sep="\t")
df_original_test

In [ ]:
df_original_test_rejected = df_original_test[df_original_test.outcome == "rejected"]

In [ ]:
drop_idx_lst = []
append_lst = []
for index, row in df_original_test_rejected.iterrows():
	coords_split = row.coords.split(":")
	chromosome = coords_split[0]
	start = int(coords_split[1].split("-")[0])-1
	end = int(coords_split[1].split("-")[1])
	strand = "+" if str(coords_split[2]) == "1" else "-"
	if df_featurized_gencode[(df_featurized_gencode.strand == strand) & (df_featurized_gencode.start == start) & (df_featurized_gencode.end == end) & (df_featurized_gencode.chr == chromosome)].empty:
		append_dict = {
			"chr": chromosome,
			"start": start,
			"end": end,
			"strand": strand,
			"Duplicate_Count": 1
		}
		append_lst.append(append_dict)
	else:
		# print(df_featurized_gencode[(df_featurized_gencode.strand == strand) & (df_featurized_gencode.start == start-1) & (df_featurized_gencode.end == end) & (df_featurized_gencode.chr == chromosome)])
		drop_idx_lst.append(index)


## These are the entries that were rejected from the original dict but accepted in the latest version

In [ ]:
df_original_test_rejected.loc[drop_idx_lst]
# chr10:72275560-72275899:1
# chr11:26567140-26569473:-1
# chr15:61861626-61890833:1
# chr15:61890863-61894121:1
# chr7:129925550-129937707:-1
# chr5:180835003-180835421:1

## We form the new Dataframe and output it to .tsv from the "still rejected" introns

In [ ]:
df_output = pd.DataFrame.from_dict(append_lst)
df_output.to_csv("data/og_rejected_introns", sep="\t", index=False)

In [ ]:
df_output